In [1]:
import pandas as pd
import praw
import os
from dotenv import load_dotenv
import boto3
import concurrent.futures
import time

In [2]:
load_dotenv()

client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')

In [3]:
# Set up S3 client with explicit credentials
s3 = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_REGION')  # Replace with your region if different
)

In [4]:
reddit = praw.Reddit(
    client_id= client_id,
    client_secret= client_secret,
    user_agent= user_agent
)

In [5]:
def fetch_comments(submission):
    submission.comments.replace_more(limit=0)
    return [
        {
            'comment': comment.body,
            'score': comment.score,
            'submission_title': submission.title,
        }
        for comment in submission.comments.list()
    ]

start_time = time.time()

subreddit_name = "soccer"
search_query = "Manchester United"
comments_data = []

# Increase the number of workers in the ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Use list comprehension to create a list of futures
    futures = [executor.submit(fetch_comments, submission) for submission in reddit.subreddit(subreddit_name).search(search_query, sort='new', limit=50)]
    
    # Use as_completed to process results as they become available
    for future in concurrent.futures.as_completed(futures):
        comments_data.extend(future.result())

# Limit to the first 100 comments
comments_data = comments_data[:100]

# Convert to DataFrame
comments_df = pd.DataFrame(comments_data)
csv_file_path = 'manchester_united_comments.csv'

# Save the comments DataFrame to CSV
comments_df.to_csv(csv_file_path, index=False)

# Upload the CSV file to S3
s3_bucket_name = 'reddit-football-text'
s3_file_name = 'man_utd.csv'

s3.upload_file(csv_file_path, s3_bucket_name, s3_file_name)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Data uploaded to S3 successfully. Time taken: {elapsed_time:.2f} seconds.")

Data uploaded to S3 successfully. Time taken: 11.76 seconds.


In [6]:
from transformers import pipeline

# Load pre-trained sentiment-analysis pipeline
sentiment_pipeline = pipeline('sentiment-analysis')

/Users/ohungchan/Documents/GitHub/Soccer-Manager-Sacking/sagemaker-venv-3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
